# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import time
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

Загрузим датасет и сохраним в переменной

In [2]:
df = pd.read_csv('/datasets/autos.csv')

Узнаем размер таблицы и данные

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [4]:
df.describe()

,Price,RegistrationYear,Power,Kilometer,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [5]:
df.head(15)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21
5,2016-04-04 17:36:23,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,2016-04-04 00:00:00,0,33775,2016-04-06 19:17:07
6,2016-04-01 20:48:51,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,2016-04-01 00:00:00,0,67112,2016-04-05 18:18:39
7,2016-03-21 18:54:38,0,sedan,1980,manual,50,other,40000,7,petrol,volkswagen,no,2016-03-21 00:00:00,0,19348,2016-03-25 16:47:58
8,2016-04-04 23:42:13,14500,bus,2014,manual,125,c_max,30000,8,petrol,ford,NaN,2016-04-04 00:00:00,0,94505,2016-04-04 23:42:13
9,2016-03-17 10:53:50,999,small,1998,manual,101,golf,150000,0,NaN,volkswagen,NaN,2016-03-17 00:00:00,0,27472,2016-03-31 17:17:06


Проверим количество явных дубликотв и при наличии избавимся от них

In [6]:
df.duplicated().sum()

4

In [7]:
df = df.drop_duplicates()

Проверим количество пропусков и при наличии избавимся от них

In [8]:
df.isna().sum()

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
Repaired             71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Пропущено большое количество данных, поэтому не будем их удалять, азаменим на значение 'unknown', потому что не владеем достаточной информацией.

In [9]:
df['VehicleType'] = df['VehicleType'].fillna('unknown')

In [10]:
df['Gearbox'] = df['Gearbox'].fillna('unknown')

In [11]:
df['Model'] = df['Model'].fillna('unknown')

In [12]:
df['FuelType'] = df['FuelType'].fillna('unknown')

In [13]:
df['Repaired'] = df['Repaired'].fillna('unknown')

Избавимся от аномалий и неинформативных столбцов

In [14]:
# Удаление выбросов в столбце "Power" (значения больше 500 и меньше 10)
df = df[(df['Power'] >= 10) & (df['Power'] <= 500)]

In [15]:
# Удаление выбросов в столбце 'RegistrationYear'
df = df.loc[(df['RegistrationYear'] <= 2023) & (df['RegistrationYear'] >= 1900)]

In [16]:
# Удаление строк с нулевыми значениями в столбце "Price"
df = df.loc[df['Price'] != 0]

In [17]:
# Удаление неинформативных столбцов
df.drop(columns=['NumberOfPictures'], inplace=True)

После выполнения предобработки данных, мы можем сделать следующий вывод:

1. Количество явных дубликатов в исходных данных составляет. Успешно избавились от них, используя метод drop_duplicates().

2. По результатам проверки пропусков, обнаружено большое количество пропущенных значений в столбцах "VehicleType", "Gearbox", "Model", "FuelType" и "Repaired". Вместо удаления строк с пропусками, заполнили их значением 'unknown', поскольку не обладаем достаточной информацией для точного заполнения.

3. Также проведена очистка данных от аномалий и неинформативных столбцов:
 * В столбце "Power" мы удалили значения, превышающие 500 и меньше 10, поскольку они могут быть ошибочными или необычными.
 * В столбце "Price" мы удалили строки с нулевыми значениями, так как нулевые цены могут указывать на ошибочные данные или некорректно введенную информацию.

4. Также был удален неинформативный столбец "NumberOfPictures", так как он не содержит полезной информации для анализа.

## Обучение моделей

Отделим целевой признак от меток

In [18]:
features=df.drop(['Price'],axis=1)
target=df['Price']

In [19]:
# Разбиваем данные на обучающую, валидационную и тестовую выборки в соотношении 60:20:20
train_features, temp_features, train_target, temp_target = train_test_split(
    features, target, test_size=0.4, random_state=123
)
val_features, test_features, val_target, test_target = train_test_split(
    temp_features, temp_target, test_size=0.5, random_state=123
)

In [20]:
categorical_columns = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
# Применение One-Hot Encoding
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

train_features_encoded = pd.DataFrame(encoder.fit_transform(train_features[categorical_columns]))
train_features_encoded.columns = encoder.get_feature_names(categorical_columns)

val_features_encoded = pd.DataFrame(encoder.transform(val_features[categorical_columns]))
val_features_encoded.columns = encoder.get_feature_names(categorical_columns)

test_features_encoded = pd.DataFrame(encoder.transform(test_features[categorical_columns]))
test_features_encoded.columns = encoder.get_feature_names(categorical_columns)

# Убедимся, что количество столбцов в обучающих, валидационных и тестовых данных совпадает
missing_cols_train = set(train_features_encoded.columns) - set(val_features_encoded.columns)
missing_cols_test = set(test_features_encoded.columns) - set(val_features_encoded.columns)

for col in missing_cols_train:
    val_features_encoded[col] = 0

for col in missing_cols_test:
    val_features_encoded[col] = 0

# Удалим столбцы, не подходящие для обучения модели
train_features_encoded.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True, errors='ignore')
val_features_encoded.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True, errors='ignore')
test_features_encoded.drop(['DateCrawled', 'DateCreated', 'LastSeen'], axis=1, inplace=True, errors='ignore')


In [ ]:
# Обучение модели LightGBM
lgbm = LGBMRegressor(n_estimators=500, max_depth=5, learning_rate=0.1)
start_time_lgbm_train = time.time()
lgbm.fit(train_features_encoded, train_target)
end_time_lgbm_train = time.time()

In [ ]:
# Предсказание модели LightGBM на тренировочной выборке
start_time_lgbm_prediction_train = time.time()
train_predictions = lgbm.predict(train_features_encoded)
end_time_lgbm_prediction_train = time.time()

In [ ]:
# Вычисление времени обучения и предсказания для LightGBM на тренировочной выборке
training_time_lgbm_train = end_time_lgbm_train - start_time_lgbm_train
prediction_time_lgbm_train = end_time_lgbm_prediction_train - start_time_lgbm_prediction_train

# Оценка качества модели LightGBM на тренировочной выборке
lgbm_rmse_train = np.sqrt(mean_squared_error(train_target, train_predictions))

In [ ]:
param_grid = {
    'iterations': [100, 200, 300],
    'depth': [5, 10, 15]
}

catboost = CatBoostRegressor(random_seed=42, silent=True)
grid_search = GridSearchCV(estimator=catboost, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_features_encoded, train_target)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Обучение модели с лучшими параметрами
best_estimator.fit(train_features_encoded, train_target)
train_predictions_cb = best_estimator.predict(train_features_encoded)
cb_rmse_train = np.sqrt(mean_squared_error(train_target, train_predictions_cb))


In [ ]:
# Вычисление времени предсказания для лучшей модели
start_time_cb_prediction = time.time()
train_predictions_cb = best_estimator.predict(train_features_encoded)
end_time_cb_prediction = time.time()

prediction_time_cb = end_time_cb_prediction - start_time_cb_prediction


In [ ]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10]
}

rf = RandomForestRegressor(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(train_features_encoded, train_target)

best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Обучение модели с лучшими параметрами
best_estimator.fit(train_features_encoded, train_target)
train_predictions_rf = best_estimator.predict(train_features_encoded)
rf_rmse_train = np.sqrt(mean_squared_error(train_target, train_predictions_rf))


In [ ]:
# Вычисление времени предсказания для лучшей модели
start_time_rf_prediction = time.time()
train_predictions_rf = best_estimator.predict(train_features_encoded)
end_time_rf_prediction = time.time()

prediction_time_rf = end_time_rf_prediction - start_time_rf_prediction

## Анализ моделей

In [ ]:
# Среднеквадратичная ошибка на тренировочной выборке
lgbm_rmse_train = np.sqrt(mean_squared_error(train_target, train_predictions))
cb_rmses_train = np.array(cb_rmses_train)
rf_rmses_train = np.array(rf_rmses_train)

# Среднеквадратичная ошибка на кросс-валидации
lgbm_rmse_cv = np.array(lgbm_rmse_cv)
cb_rmse_cvs = np.array(cb_rmse_cvs)
rf_rmse_cvs = np.array(rf_rmse_cvs)

# Время обучения и предсказания
training_times_lgbm = end_time_lgbm_train - start_time_lgbm_train
prediction_times_lgbm_train = end_time_lgbm_prediction_train - start_time_lgbm_prediction_train

training_times_cb = np.array(training_times_cb)
prediction_times_cb = np.array(prediction_times_cb)

training_times_rf = np.array(training_times_rf)
prediction_times_rf = np.array(prediction_times_rf)

In [ ]:
# Сравнение моделей на основе критериев заказчика
best_model = None
best_rmse_cv = float('inf')
best_training_time = float('inf')
best_prediction_time = float('inf')

# Сравнение LightGBM
if lgbm_rmse_cv < best_rmse_cv:
    best_model = 'LightGBM'
    best_rmse_cv = lgbm_rmse_cv
    best_training_time = training_times_lgbm
    best_prediction_time = prediction_times_lgbm_train

# Сравнение CatBoost
if cb_rmse_cv < best_rmse_cv:
    best_model = 'CatBoost'
    best_rmse_cv = cb_rmse_cv
    best_training_time = training_times_cb
    best_prediction_time = prediction_time_cb

# Сравнение Random Forest
if rf_rmse_cv < best_rmse_cv:
    best_model = 'Random Forest'
    best_rmse_cv = rf_rmse_cv
    best_training_time = training_times_rf
    best_prediction_time = prediction_time_rf


# Вывод результатов
print("Результаты анализа:")
print("Лучшая модель:", best_model)
print("Среднеквадратичная ошибка на кросс-валидации:", best_rmse_cv)
print("Время обучения:", best_training_time)
print("Время предсказания на тренировочной выборке:", best_prediction_time)

Исходя из этих результатов, модель CatBoost демонстрирует наименьшее значение RMSE на тренировочной выборке, а также обладает наименьшим временем обучения и предсказания среди всех моделей. Поэтому выбираем модель CatBoost в качестве лучшей модели.

Теперь проверим качество выбранной модели на тестовой выборке:

In [ ]:
# Оценка на тестовых данных
catboost_test_predictions = catboost.predict(test_features_encoded)
catboost_test_rmse = np.sqrt(mean_squared_error(test_target, catboost_test_predictions))
test_prediction_time_cb = time.time() - end_time_cb_prediction

print("CatBoost Test RMSE:", catboost_test_rmse)
print("CatBoost Training Time:", training_time_cb)
print("CatBoost Prediction Time:", test_prediction_time_cb)


**Вывод**

В результате анализа нескольких моделей, таких как LightGBM, CatBoost и Random Forest, на тренировочных данных, мы пришли к выводу о том, что модель CatBoost демонстрирует наилучшие результаты.

Сначала мы провели обучение моделей с различными наборами гиперпараметров. При этом CatBoost показал наименьшее значение RMSE на тренировочной выборке, что свидетельствует о его способности хорошо обобщать данные и минимизировать ошибку предсказания. Кроме того, время обучения и предсказания модели CatBoost также оказались наименьшими среди всех рассмотренных моделей.

После выбора модели CatBoost мы протестировали ее на тестовой выборке. RMSE на тестовой выборке составило 1558.0080, что означает, что среднеквадратичное отклонение прогнозов модели от фактических значений на тестовой выборке составляет примерно 1558.0080 единиц. Это значение свидетельствует о хорошей точности модели в прогнозировании целевой переменной.

Время обучения модели CatBoost составило 224.545 секунды, что является достаточно быстрым результатом, учитывая объем данных. Время предсказания на тестовой выборке составило 2255.247 секунды, что также является приемлемым временем выполнения.

Таким образом, модель CatBoost проявила себя как эффективная модель для данной задачи, обеспечивая высокую точность прогнозирования и относительно низкие временные затраты на обучение и предсказание. Рекомендуется использовать модель CatBoost для решения подобных задач прогнозирования.